In [22]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import os, time, uuid

In [23]:
import yaml
with open('../config.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [24]:
ENDPOINT = f"https://{config['custom_vision']['endpoint']}/"
training_key = config["custom_vision"]["training_key"]
prediction_key = config["custom_vision"]["prediction_key"]
prediction_resource_id = config["custom_vision"]["prediction_resource_id"]
firenet_tags = config["firenet_tags"]


In [25]:
publish_iteration_name = "FireNet"

credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

In [26]:
if not config["project_id"]:
    # Create a new project
    print("Creating project...")
    obj_detection_domain = next(
        domain
        for domain in trainer.get_domains()
        if domain.type == "ObjectDetection" and domain.name == "General"
    )

    # Use uuid to avoid project name collisions.
    project = trainer.create_project(
        str(uuid.uuid4()),
        domain_id=obj_detection_domain.id,
        classification_type="Multiclass",
    )
else:
    project = trainer.get_project(config["project_id"])

f"Project ID: {project.id}"


Creating project...


'Project ID: 591faad1-d097-4f4d-bccb-53872cd6c9ee'

## Upload and tag images

In [27]:
import pandas as pd
from pathlib import Path

firenet_path = Path(config["firenet_path"])
images_path = firenet_path / "Images"

df = pd.read_pickle(firenet_path / "object_detection.pkl")
df.head()

,filename,tag,left,top,width,height,truncated,difficult
0,0000.jpg,Fire_Extinguisher,0.172526,0.497070,0.322917,0.455078,0,0
1,0000.jpg,Fire_Suppression_Signage,0.322266,0.377441,0.128255,0.090332,0,0
2,0001.jpg,Fire_Blanket,0.236613,0.544428,0.298193,0.298005,0,0
3,0001.jpg,Fire_Extinguisher,0.579317,0.637613,0.367805,0.336596,0,0
4,0001.jpg,Fire_Suppression_Signage,0.380857,0.027673,0.326640,0.298005,0,0


### Add tags to the project

In [28]:
tags = {}
for az_tag in trainer.get_tags(project.id):
    if az_tag.name in firenet_tags.values():
        tags[az_tag.name] = az_tag
for tag_name in df.tag.unique():
    if tag_name in firenet_tags and tag_name not in tags:
        tags[firenet_tags[tag_name]] = trainer.create_tag(project.id, firenet_tags[tag_name])
len(tags)

1

### Adding images

In [29]:
from tqdm import tqdm

batch_size = config["upload_batch_size"]
tagged_images_with_regions = []

print("Loading images and labels...")

images = df.filename.unique()
i = 0
for file_name in tqdm(images):
    i += 1

    rows = df.where(df.filename == file_name).dropna()
    regions = []
    for _, row in rows.iterrows():
        if row["tag"] in firenet_tags:
            regions.append(
                Region(
                    tag_id=tags[firenet_tags[row["tag"]]].id,
                    left=row["left"],
                    top=row["top"],
                    width=row["width"],
                    height=row["height"],
                )
            )
    if regions:
        with open(images_path / file_name, mode="rb") as image_contents:
            tagged_images_with_regions.append(
                ImageFileCreateEntry(
                    name=file_name, contents=image_contents.read(), regions=regions
                )
            )

    if i % batch_size == 0 and tagged_images_with_regions:
        print(f"Uploading image batch {i} ...")
        upload_result = trainer.create_images_from_files(
            project.id, ImageFileCreateBatch(images=tagged_images_with_regions)
        )
        tagged_images_with_regions = []
        if not upload_result.is_batch_successful:
            errors = False
            for image in upload_result.images:
                if image.status != "OK" and image.status != "OKDuplicate":
                    errors = True
                    print(f"Image {image.source_url}: {image.status}")
            if errors:
                print(f"Image batch {i} upload failed.")
                break

if len(tagged_images_with_regions) > 0:
    print(f"Uploading latest batch {i}...")
    upload_result = trainer.create_images_from_files(
        project.id, ImageFileCreateBatch(images=tagged_images_with_regions)
    )
    tagged_images_with_regions = []
    if not upload_result.is_batch_successful:
        errors = False
        for image in upload_result.images:
            if image.status != "OK" and image.status != "OKDuplicate":
                errors = True
                print(f"Image {image.source_url}: {image.status}")
        if errors:
            print(f"Latest Image batch upload failed.")


Loading images and labels...


  3%|▎         | 42/1449 [00:00<00:06, 204.55it/s]

Uploading image batch 50 ...


  6%|▌         | 89/1449 [00:05<01:33, 14.53it/s]

Uploading image batch 100 ...


...

Uploading image batch 1350 ...


 96%|█████████▌| 1386/1449 [01:37<00:03, 20.14it/s]

Uploading image batch 1400 ...


100%|██████████| 1449/1449 [01:42<00:00, 14.11it/s]


Uploading latest batch 1449...
